In [1]:
from utils.find_miner_id import ReadTokens

In [2]:
reader = ReadTokens()

In [3]:
wallet = '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn'
data = reader.get_latest_miner_id(wallet)
description = reader.get_token_description(data['tokenId'])
description

{'address': '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn',
 'height': 1324551,
 'minimumPayout': 2,
 'tokens': [{'token': 'Bober', 'value': 20},
  {'token': 'RSN', 'value': 20},
  {'token': 'ERGO', 'value': 60}],
 'season': 0,
 'type': 'Miner ID',
 'fan_club': 'QX'}

In [4]:
possible_tokens = reader.find_token_name_in_wallet(wallet, 'Sigmanaut Mining Pool Miner ID - Season 0')
possible_tokens

[{'tokenId': 'aa11470f16d9f13ed3073f497d7934a390b22a1400c8acead209406db38eb68a',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': '8d319044d5a30ec285e74620ee67aa64be224f58cccd8f050e921d55637144a3',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': '48c440d82c7749dbaa5ab8b6169f57d1ddafdf4e2f9380f4aad1aca12c64442e',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': 'a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': '270619a2c297c0818e8afc59c8a0ae679fed0c3d91d9295384f4203f23db651a',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}]

In [16]:
my_tokens = []
miner_id = None
earliest_height = 0
for token in possible_tokens:
    
    id = token['tokenId']
    print(id, 'id')
    base_url = 'https://api.ergoplatform.com/api/v1/boxes/byTokenId'
    url = '{}/{}'.format(base_url, id)
    data = reader.get_api_data(url)
    oldest_height = 1e30
    # for a given token we want to find the earliest TX with it so we can see who minted it
    for item in data['items']:
        if oldest_height > item['creationHeight']:
            oldest_height = item['creationHeight']
            # ls = item['boxId'], item['creationHeight'], item['transactionId'], item['address']
            tx = item['transactionId']
    
    # tx = ls[2]
    wallet_minted_tokens = []
    url = 'https://api.ergoplatform.com/api/v1/transactions'
    tx_url = '{}/{}'.format(url, tx)
    data = reader.get_api_data(tx_url)
    # print(data)
    box_id_first_input = data['inputs'][0]['boxId']
    outputs = data['outputs']
    # print(outputs, 'oi')
    
    for sample in outputs:
        try:   
            assets = sample['assets'][0]
            if assets['tokenId'] == box_id_first_input:
                minting_address = sample['address']
                if wallet == minting_address:
                    if sample['creationHeight'] > earliest_height:
                        oldest_height = sample['creationHeight']
                        my_tokens.append(token)
                        miner_id = token
                        print('breakk', miner_id['tokenId'], oldest_height)
                        break
        except IndexError:
            pass

aa11470f16d9f13ed3073f497d7934a390b22a1400c8acead209406db38eb68a id
breakk aa11470f16d9f13ed3073f497d7934a390b22a1400c8acead209406db38eb68a 1324410
8d319044d5a30ec285e74620ee67aa64be224f58cccd8f050e921d55637144a3 id
breakk 8d319044d5a30ec285e74620ee67aa64be224f58cccd8f050e921d55637144a3 1308723
48c440d82c7749dbaa5ab8b6169f57d1ddafdf4e2f9380f4aad1aca12c64442e id
breakk 48c440d82c7749dbaa5ab8b6169f57d1ddafdf4e2f9380f4aad1aca12c64442e 1290168
a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e id
breakk a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e 1356351
270619a2c297c0818e8afc59c8a0ae679fed0c3d91d9295384f4203f23db651a id
breakk 270619a2c297c0818e8afc59c8a0ae679fed0c3d91d9295384f4203f23db651a 1324551


In [17]:
miner_id

{'tokenId': '270619a2c297c0818e8afc59c8a0ae679fed0c3d91d9295384f4203f23db651a',
 'amount': 1,
 'decimals': 0,
 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}

In [18]:
my_tokens = []
most_recent_token = None
highest_creation_height = 0

for token in possible_tokens:
    token_id = token['tokenId']
    print(token_id, 'id')
    
    # Get boxes by token ID
    boxes_url = f'https://api.ergoplatform.com/api/v1/boxes/byTokenId/{token_id}'
    boxes_data = reader.get_api_data(boxes_url)
    
    # Find the earliest transaction for this token
    earliest_tx = min(boxes_data['items'], key=lambda x: x['creationHeight'])
    earliest_tx_id = earliest_tx['transactionId']
    
    # Get transaction details
    tx_url = f'https://api.ergoplatform.com/api/v1/transactions/{earliest_tx_id}'
    tx_data = reader.get_api_data(tx_url)
    
    box_id_first_input = tx_data['inputs'][0]['boxId']
    
    # Check outputs for minting information
    for output in tx_data['outputs']:
        assets = output.get('assets', [])
        if assets and assets[0]['tokenId'] == box_id_first_input:
            minting_address = output['address']
            creation_height = output['creationHeight']
            
            if minting_address == wallet:
                my_tokens.append(token)
                if creation_height > highest_creation_height:
                    highest_creation_height = creation_height
                    most_recent_token = token
            break  # Found the minting output for this token, move to next token

print(f"Most recent token: {most_recent_token}")
print(f"All tokens minted by the wallet: {my_tokens}")
print(f"Number of tokens minted by the wallet: {len(my_tokens)}")

aa11470f16d9f13ed3073f497d7934a390b22a1400c8acead209406db38eb68a id
8d319044d5a30ec285e74620ee67aa64be224f58cccd8f050e921d55637144a3 id
48c440d82c7749dbaa5ab8b6169f57d1ddafdf4e2f9380f4aad1aca12c64442e id
a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e id
270619a2c297c0818e8afc59c8a0ae679fed0c3d91d9295384f4203f23db651a id
Most recent token: {'tokenId': 'a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e', 'amount': 1, 'decimals': 0, 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}
All tokens minted by the wallet: [{'tokenId': 'aa11470f16d9f13ed3073f497d7934a390b22a1400c8acead209406db38eb68a', 'amount': 1, 'decimals': 0, 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}, {'tokenId': '8d319044d5a30ec285e74620ee67aa64be224f58cccd8f050e921d55637144a3', 'amount': 1, 'decimals': 0, 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}, {'tokenId': '48c440d82c7749dbaa5ab8b6169f57d1ddafdf4e2f9380f4aad1aca12c64442e', 'amount': 1, 'decimals': 0, 'name': 'Sigma

In [19]:
most_recent_token

{'tokenId': 'a8939dbf45202a30a9f098747e04a082dcacceb222c60d671dde721cff85991e',
 'amount': 1,
 'decimals': 0,
 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}